# Обучение без учителя

Рассмотрим простейший алгоритм обучения без учителя - кластеризацию методом к-средних (k-means)


In [ ]:
# подготовка среды

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# настройки картинок
import seaborn as sns; sns.set()

# Введение в K-means

Кластеризацией данных будем неформально называть процедуру группировки объектов в классы так, что объекты внутри класса были близки друг к другу на основе некоторой введенной нами метрики (Евклидово расстояние, семантическая похожесть текстов, цвет картинок, и т.д.).

Алгоритм работает без учителя, то есть нам необходима матрица объекты-признаки, а так же желаемое число классов.

Создадим некоторый исcкуственный дата сет, в котором явно присутствуют классы объектов, но эти классы нам не известны заранее:

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], s=50);

Применить k-means к этим данным значит назначить каждому объекту некоторый класс (цвет). 
Используется эффективный алгоритм максимизации ожидания *Expectation Maximization (EM)*:

In [ ]:
from sklearn.cluster import KMeans
est = KMeans(4)  # 4 класса
est.fit(X)
y_kmeans = est.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='rainbow');

## Подробнее об алгоритме K-Means: Expectation Maximization

1. Начинаем с случайных центров классов
2. Повторять до сходимости:
   A. Назначить всем точкам класс, выбирая центы классов с минимальным расстоянием
   B. Пересчитать координату центра класса 
   
Визуально это выглядит так: https://www.youtube.com/watch?time_continue=50&v=5I3Ei69I40s

### Проблемы KMeans 

K-means не гарантирует нахождение глобального оптимального решения. К тому же, как видно из описания, алгоритм начинается со случайной инициализации центров классов. В результате алгоритм при разных запусках находит разное решение.

На практике делают несколько запусков кластеризации, затем выбирая лучший результат.

## Кластеризация цифр с помощью KMeans

Запустим алгорит K-means на картинках. Найдем 10 классов, используя Евклидову меру в 64 мерном пространстве признаков (корень из суммы квадратов разницы яркости пикселей)

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
est = KMeans(n_clusters=10)
clusters = est.fit_predict(digits.data)
est.cluster_centers_.shape

Теперь мы можем построить центры (64 значения) каждого из 10 классов:

In [ ]:
fig = plt.figure(figsize=(8, 3))
for i in range(10):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.imshow(est.cluster_centers_[i].reshape((8, 8)), cmap=plt.cm.binary)

Как видно, некоторые классы вполне себе различимы.

# ЗАДАНИЕ

Посмотрите, что произойдет при увеличении числа классов? Какие центроиды получились?


## Еще пример: KMeans для компрессии цвета

In [ ]:
from sklearn.datasets import load_sample_image
china = load_sample_image("china.jpg")
plt.imshow(china)
plt.grid(False);

Картинка хранится в 3-х мерном массиве размера ``(высота, ширина, RGB)``:

In [ ]:
china.shape

Как и в примере с цифрами, преобразуем в одномерный массив:

In [ ]:
X = (china / 255.0).reshape(-1, 3)
print(X.shape)

Теперь у нас 273,280 точки в 3 измерениях.

Задача - сжать $256^3$ цветов в меньшее количество (например 64). По сути, мы хотим найти кластеры из N цветов и создать новое изображение, в котором цвет будет заменяться цветом центра ближайшего класса 


Для быстроты и возможности работы с большими данными, используем ``MiniBatchKMeans``

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
# уменьшаем до 64 цветов
n_colors = 64

X = (china / 255.0).reshape(-1, 3)
    
model = MiniBatchKMeans(n_colors)
labels = model.fit_predict(X)
colors = model.cluster_centers_
new_image = colors[labels].reshape(china.shape)
new_image = (255 * new_image).astype(np.uint8)

# создаем и рисуем
with sns.axes_style('white'):
    plt.figure()
    plt.imshow(china)
    plt.title('input: 16 million colors')

    plt.figure()
    plt.imshow(new_image)
    plt.title('{0} colors'.format(n_colors))